In [ ]:
import socket
import threading

class Server:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.clients = []
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server_socket.bind((self.host, self.port))

    def start(self):
        self.server_socket.listen(5)
        print(f"Server listening on {self.host}:{self.port}")
        while True:
            client_socket, client_address = self.server_socket.accept()
            print(f"New connection from {client_address}")
            client_thread = threading.Thread(target=self.handle_client, args=(client_socket, client_address))
            client_thread.start()

    def handle_client(self, client_socket, client_address):
        self.clients.append(client_socket)
        while True:
            try:
                message = client_socket.recv(1024).decode("utf-8")
                if not message:
                    break
                print(f"From {client_address}: {message}")
                self.broadcast(message, client_socket)
            except ConnectionResetError:
                break
        print(f"Connection with {client_address} closed.")
        self.clients.remove(client_socket)
        client_socket.close()

    def broadcast(self, message, sender_socket):
        for client_socket in self.clients:
            if client_socket != sender_socket:
                try:
                    client_socket.send(message.encode("utf-8"))
                except ConnectionResetError:
                    continue

class Client:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client_socket.connect((self.host, self.port))
        self.receive_thread = threading.Thread(target=self.receive_messages)
        self.receive_thread.start()

    def send_message(self, message):
        self.client_socket.send(message.encode("utf-8"))

    def receive_messages(self):
        while True:
            try:
                message = self.client_socket.recv(1024).decode("utf-8")
                print(message)
            except ConnectionResetError:
                print("Connection to the server closed.")
                break

def main():
    mode = input("Choose mode (server/client): ").lower()
    if mode == "server":
        host = input("Enter server IP address: ")
        port = int(input("Enter server port: "))
        server = Server(host, port)
        server.start()
    elif mode == "client":
        host = input("Enter server IP address: ")
        port = int(input("Enter server port: "))
        client = Client(host, port)
        while True:
            message = input()
            if message.lower() == "quit":
                break
            client.send_message(message)
    else:
        print("Invalid mode. Choose 'server' or 'client'.")

if __name__ == "__main__":
    main()
